# **Projeto IA - Connect 4**

### Membros do Grupo:
- Maximiliano Vítor Phillips e Sá (up202305979)
- Orlando Miguel Carvalho-Soares (up202303606)
- Rui Jorge Pereira Rua (up202305259)

## Índice
1. Introdução
2. Código Universal
3. Game Modes
    - Humano x Humano
    - Humano x Computador
    - Computador x Computador


## 1. Introdução

Este projeto tem como objetivo desenvolver e implementar um programa capaz de jogar 'Connect-4' com um humano. O programa tem 4 modos: Humano-Humano, Humano-Computador(MCTS), Humano-Computador(Árvores de Decisão) e Computador-Computador. O programa utiliza os algoritmos MCTS e Árvores de Decisão usando o procedimento ID3 no modo Computador-Computador.


## 2. Código Universal

O seguinte código é 'universal' e serve para gerir o estado atual do jogo.

meta.py serve para guardar as definições básicas do jogo e o valor C para o algoritmo MCTS.

ConnectState.py guarda o estado do jogo e as funções básicas para UI output e para verificar se existe jogadas legais e se existe uma vitória. 

meta.py

In [ ]:
import math

#meta dados relativos ao jogo
class GameMeta:
    PLAYERS = {'none': 0, 'one': 1, 'two': 2}
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    INF = float('inf')
    ROWS = 6
    COLS = 7
    GAMEMODE = None

#valor C do UCB
class MCTSMeta:
    C = math.sqrt(2)



ConnectState.py

In [ ]:
from copy import deepcopy
import numpy as np
from meta import GameMeta


class ConnectState:
    '''
        Construtor do board e de alguns metadados sobre o estado atual. 
        Ex: jogador atual, jogada anterior, altura de cada coluna (height[col] = 0 -> coluna col cheia)
    '''

    def __init__(self):
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        self.to_play = GameMeta.PLAYERS['one']
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        self.last_played = []

    #retorna uma cópia do estado atual
    def get_board(self):
        return deepcopy(self.board)
    
    #dá print_board ao board atual na consola
    def print_board(self):
        print('=============================')

        for row in range(GameMeta.ROWS):
            for col in range(GameMeta.COLS):
                print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
            print('|')

        print('=============================')
        print('  1   2   3   4   5   6   7')

    #marca a jogada escolhida pelo jogador no board, e dá update a alguns metadados do estado atual
    def move(self, col):
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

    #retorna uma lista com as jogadas que ainda não estão cheias (height[col] != 0)
    def get_legal_moves(self):
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

    #verfica se há vencedor e retorna o seu número (X = 1 ; O = 2), se não houver retorna 0
    def check_win(self):
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0

    def check_win_from(self, row, col):
        player = self.board[row][col]
        """
        Ultima jogada realizada está na posição (row, col)
        Verificar a grid 6x7 à volta à procura de um vencedor
        """

        consecutive = 1
        # verificar horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # verificar vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal1
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal2
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False

    #verifica se o jogo já acabou verificando se há vencedor ou se ainda existem jogadas possíveis
    def game_over(self):
        return self.check_win() or len(self.get_legal_moves()) == 0

    #verifica quem ganhou ou se é empate
    def get_result(self):
        winner = self.check_win()
        if len(self.get_legal_moves()) == 0 and winner == 0:
            return GameMeta.OUTCOMES['draw']
        return GameMeta.OUTCOMES['one'] if winner == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']



## 3. Game Modes

Existem 4 modos de jogo:
1. Humano x Humano
2. Humano x Computador (MCTS)
3. Humano x Computador (Árvores de Decisão)
4. Computador x Computador (MCTS x Árvores de Decisão)

### Código Base para a Interface

game.py

In [ ]:
from ConnectState import ConnectState
from mcts import MCTS
from meta import GameMeta
from tree import trainTree
import numpy as np
import random

def playPvP():
    state = ConnectState()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Player 1 Move ---
        while True:
            p1_input = input("Player 1 ('X') enter a move: ")
            if not p1_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p1Move = int(p1_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p1Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p1Move)
        state.print_board()

        if state.game_over():
            print("Player one ('X') won!")
            break

        # --- Player 2 Move ---
        while True:
            p2_input = input("Player 2 ('O') enter a move: ")
            if not p2_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p2Move = int(p2_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p2Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p2Move)
        state.print_board()

        if state.game_over():
            print("Player two ('O') won!")
            break


def playPvC1():
    state = ConnectState()
    mcts = MCTS(state)
    changed_To_Attack = False # se o valor de C foi mudado ou não
    num_Plays = 0

    while not state.game_over():
        print("Current state:")
        state.print_board()

        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)
        mcts.move(user_move)

        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        print("Thinking...")

        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")
        move = mcts.best_move()
        state.move(move)
        mcts.move(move)

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True

def playPvC2():
    state = ConnectState()
    tree = trainTree()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)

        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        new_state = np.array(state.get_board()).flatten()

        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()

        if predicted_move not in legal_moves:
            state.move(legal_moves[random.randint(0, len(legal_moves)-1)])
        else:
            state.move(predicted_move)

        print("Tree chose to play in column: " + str(predicted_move+1))

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

def playCvC():
    state = ConnectState()
    tree = trainTree()
    mcts = MCTS(state)
    changed_To_Attack = False
    num_Plays = 0
    print("Current state:")
    state.print_board()

    while not state.game_over():
        # --- MCTS turn ---
        print("Thinking (MCTS)...")
        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")

        mcts_move = mcts.best_move()
        state.move(mcts_move)
        mcts.move(mcts_move)

        print("MCTS chose column:", mcts_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("MCTS ('X') won!")
            else:
                print("Draw!")
            break

        # --- Tree turn ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()
        if predicted_move not in legal_moves:
            # fallback to random legal move
            predicted_move = random.choice(legal_moves)

        state.move(predicted_move)
        mcts.move(predicted_move)

        print("Tree chose column:", predicted_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Tree ('O') won!")
            else:
                print("Draw!")
            break

        # Optionally adjust MCTS’s C parameter after a number of full turns
        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True
    


'''
    Inicializador do modo de jogo pretendido:
    1. PvP
    2. PvC (mcts)
    3. PvC (tree)
    4. CvC
'''
if __name__ == "__main__":
    print("Welcome to Connect 4!\nPlease select one of the following game modes by selecting its corresponding number:\n1. Human vs. Human\n2. Human vs. Computer(mcts)\n3. Human vs. Computer(tree)\n4. Computer vs. Computer")
    GameMeta.GAMEMODE = int(input("Select option:"))
    while 1 > GameMeta.GAMEMODE or GameMeta.GAMEMODE > 4:
        print("Invalid Gamemode")
        GameMeta.GAMEMODE = int(input("Select option:"))
    if GameMeta.GAMEMODE == 1:
        playPvP()
    elif GameMeta.GAMEMODE == 2:
        playPvC1()
    elif GameMeta.GAMEMODE == 3:
        playPvC2()
    else:
        playCvC()
